In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import Normalizer , scale
from sklearn.model_selection import GridSearchCV , KFold , cross_val_score
from sklearn.metrics import mean_squared_log_error,mean_squared_error, r2_score,mean_absolute_error 
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from sklearn.utils import shuffle
from keras.utils import plot_model
%matplotlib inline

In [2]:
checkin_cols = ['user_id', 'poi_id', 'timestamp', 'timezone']
checkins = pd.read_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\TIST2015_Checkins.csv', sep=',', names=checkin_cols, encoding='latin-1')

venue_cols = ['poi_id', 'latitude', 'longitude', 'category', 'country_code']
pois = pd.read_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\TIST2015_POIs.csv', sep=',', names=venue_cols, encoding='latin-1')

databasepoi_cols = ['poi_id']
databasepoi = pd.read_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\databasePOI.csv', sep=',', names=databasepoi_cols, encoding='latin-1')[1:]

#Getting all the pois from the database (Der skal være cirka 91000?)
pois_subset = pois.merge(databasepoi, on='poi_id')

#Getting checkins from subset (Der skal være cirka 800000)
temp = checkins.copy()
checkins_subset = temp.merge(pois_subset, on='poi_id')

#Getting users from subset (Der skal cirka være 172000)
users_subset = checkins_subset.copy()
users_subset.drop_duplicates(subset="user_id", keep = 'first', inplace = True)

C:\Users\lasse\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (0,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\lasse\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [17]:
#Partition 1
checkin_data = users_subset[:28741]
df = checkin_data.set_index('user_id').poi_id.str.get_dummies(',')
df = df.groupby('user_id').max()

In [19]:
checkin_data_no_duplicates = checkin_data.copy()
checkin_data_no_duplicates.drop_duplicates(subset ="poi_id",
                     keep = 'first', inplace = True)
checkin_data_no_duplicates = pd.DataFrame(checkin_data_no_duplicates, columns = ['poi_id', 'category'])

In [20]:
#Extract categorical data
print("Extract categorical data")
categories = pd.DataFrame(checkin_data, columns=['category'])
categories.drop_duplicates(subset ="category",
                           keep = 'first', inplace = True)
category_length = len(categories)
categories_numpy = categories.to_numpy()

Extract categorical data


In [21]:
#Extracting all of the users and the pois
listofusers = pd.DataFrame(checkin_data, columns= ['user_id']).groupby('user_id').max().sample(frac=1)
listofpois = pd.DataFrame(checkin_data, columns= ['poi_id', 'latitude', 'longitude']).groupby('poi_id').max().sample(frac=1)
userarray = listofusers.index.to_numpy()
poiarray = listofpois.index.to_numpy()
userdataframe = pd.DataFrame(userarray, columns = ['Users'])
poidataframe = pd.DataFrame(poiarray, columns = ['Poi'])
dot = userdataframe.merge(poidataframe, how='cross')

MemoryError: Unable to allocate 2.82 GiB for an array with shape (378782032,) and data type int64

In [ ]:
rows_list = []
for i in range(len(dot)):
    temp = dot.loc[i, "Poi"]
    latitude = listofpois.loc[temp]['latitude']
    longitude = listofpois.loc[temp]['longitude']
    dict1 = {'latitude':latitude, 'longitude':longitude}
    rows_list.append(dict1)
    #latitude = poiarray[i]
latlong = pd.DataFrame(rows_list)

In [ ]:
#Creating dataset
userdot = pd.DataFrame(dot, columns= ['Users'])
latlong['latitude'] = pd.to_numeric(latlong['latitude'])
latlong['longitude'] = pd.to_numeric(latlong['longitude'])
dataset = pd.concat([userdot, latlong], axis=1)

In [ ]:
#Extracting ground_truth from incidence matrix
rows_list = []
category_list = []
for i in range(len(dot)):
    temp = df[dot.loc[i, "Poi"]][dot.loc[i, "Users"]]
    dict1 = {'ground_truth':float(temp)}
    rows_list.append(dict1)
    #Extract category from the list
    category = checkin_data_no_duplicates.loc[checkin_data_no_duplicates['poi_id'] == dot.loc[i, "Poi"]]
    cat = category['category']
    index = np.where(categories_numpy == [cat])[0][0]
    category_list.append(index)
#category_label = 
groundtruth = pd.DataFrame(rows_list)
#result = pd.concat([dot, groundtruth], axis=1)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import Normalizer , scale
from sklearn.model_selection import GridSearchCV , KFold , cross_val_score
from sklearn.metrics import mean_squared_log_error,mean_squared_error, r2_score,mean_absolute_error 
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from sklearn.utils import shuffle
%matplotlib inline

checkin_cols = ['user_id', 'poi_id', 'timestamp', 'timezone']
checkins = pd.read_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\TIST2015_Checkins.csv', sep=',', names=checkin_cols, encoding='latin-1')

venue_cols = ['poi_id', 'latitude', 'longitude', 'category', 'country_code']
pois = pd.read_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\TIST2015_POIs.csv', sep=',', names=venue_cols, encoding='latin-1')

databasepoi_cols = ['poi_id']
databasepoi = pd.read_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\databasePOI.csv', sep=',', names=databasepoi_cols, encoding='latin-1')[1:]

#Getting all the pois from the database (Der skal være cirka 91000?)
pois_subset = pois.merge(databasepoi, on='poi_id')

#Getting checkins from subset (Der skal være cirka 800000)
temp = checkins.copy()
checkins_subset = temp.merge(pois_subset, on='poi_id')

#Getting users from subset (Der skal cirka være 172000)
users_subset = checkins_subset.copy()
users_subset.drop_duplicates(subset="user_id", keep = 'first', inplace = True)

#Partition 1
print("Partition 1")
checkin_data = users[:57482]
df = checkin_data.set_index('user_id').poi_id.str.get_dummies(',')
df = df.groupby('user_id').max()

checkin_data_no_duplicates = checkin_data.copy()
checkin_data_no_duplicates.drop_duplicates(subset ="poi_id",
                     keep = 'first', inplace = True)
checkin_data_no_duplicates = pd.DataFrame(checkin_data_no_duplicates, columns = ['poi_id', 'category'])

#Extract categorical data
print("Extract categorical data")
categories = pd.DataFrame(checkin_data, columns=['category'])
categories.drop_duplicates(subset ="category",
                           keep = 'first', inplace = True)
category_length = len(categories)
categories_numpy = categories.to_numpy()

#Extracting all of the users and the pois
print("Extracting all of the users and the pois")
listofusers = pd.DataFrame(checkin_data, columns= ['user_id']).groupby('user_id').max().sample(frac=1)
listofpois = pd.DataFrame(checkin_data, columns= ['poi_id', 'latitude', 'longitude']).groupby('poi_id').max().sample(frac=1)
userarray = listofusers.index.to_numpy()
poiarray = listofpois.index.to_numpy()
userdataframe = pd.DataFrame(userarray, columns = ['Users'])
poidataframe = pd.DataFrame(poiarray, columns = ['Poi'])
dot = userdataframe.merge(poidataframe, how='cross')

#LatLong
print("LatLong")
rows_list = []
for i in range(len(dot)):
    temp = dot.loc[i, "Poi"]
    latitude = listofpois.loc[temp]['latitude']
    longitude = listofpois.loc[temp]['longitude']
    dict1 = {'latitude':latitude, 'longitude':longitude}
    rows_list.append(dict1)
    #latitude = poiarray[i]
latlong = pd.DataFrame(rows_list)

#Creating dataset
print("Creating dataset")
userdot = pd.DataFrame(dot, columns= ['Users'])
latlong['latitude'] = pd.to_numeric(latlong['latitude'])
latlong['longitude'] = pd.to_numeric(latlong['longitude'])
dataset = pd.concat([userdot, latlong], axis=1)

#Extracting ground_truth from incidence matrix
print("Extracting ground_truth from incidence matrix")
rows_list = []
category_list = []
for i in range(len(dot)):
    temp = df[dot.loc[i, "Poi"]][dot.loc[i, "Users"]]
    dict1 = {'ground_truth':float(temp)}
    rows_list.append(dict1)
    #Extract category from the list
    category = checkin_data_no_duplicates.loc[checkin_data_no_duplicates['poi_id'] == dot.loc[i, "Poi"]]
    cat = category['category']
    index = np.where(categories_numpy == [cat])[0][0]
    category_list.append(index)
#category_label = 
groundtruth = pd.DataFrame(rows_list)
#result = pd.concat([dot, groundtruth], axis=1)

categories = pd.DataFrame(category_list, columns=['category'])
datasetst = pd.concat([dataset, categories], axis=1)

dataset_numpy = datasetst.to_numpy()
labels_numpy = groundtruth.to_numpy()
categories_numpy = categories.to_numpy()
x_train, x_test, y_train, y_test = train_test_split(dataset_numpy, labels_numpy, test_size=0.05, random_state=0)

x_train_df = pd.DataFrame(x_train, columns=['User','Latitude','Longitude', '0'])
x_test_df = pd.DataFrame(x_test, columns=['User','Latitude','Longitude', '0'])
y_train_df = pd.DataFrame(y_train)
y_test_df = pd.DataFrame(y_test)

#Dataset with Users
dataset1_df = pd.DataFrame(x_train_df['User'])

#Dataset with Poi's
dataset2_df = pd.DataFrame(x_train_df[['Latitude', 'Longitude']])

dataset3_df = pd.DataFrame(x_train_df[['0']])

dataset1_df.to_csv(sep=',', index=['User'])
dataset2_df.to_csv(sep=',', index=['Latitude', 'Longitude'])
dataset3_df.to_csv(sep=',', index=['0'])